In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb


In [78]:
# import cleaned training data
# convert dates to datetime object, use dates as index, sort by date
df = pd.read_csv("train_clean.csv", index_col=[0], parse_dates=[0], dayfirst=True)

df = df.sort_index()
df.tail()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
date,,,,,
2015-10-31,33,50,5003,3999.0,1.0
2015-10-31,33,26,3353,3999.0,1.0
2015-10-31,33,50,4872,1499.0,2.0
2015-10-31,33,25,13240,450.0,2.0
2015-10-31,33,19,20949,5.0,2.0


In [70]:
# use 2nd to last full month (august 2015) as validation
# last full month (september 2015) as test

val_start_date, test_start_date, date_cutoff = '2015-01-08', '2015-01-09', '2015-01-10'

train = df[df.index < val_start_date]

train.tail(20)

TypeError: '<' not supported between instances of 'numpy.ndarray' and 'numpy.ndarray'